<img src="Figs/Banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657 Microwave Remote Sensing <b> </font>

<font size="5"> <b> Lab 9: InSAR Time Series Analysis using MintPy and ARIA products</b> </font>
<br> <img src="Figs/NASALogo.png" width="250" align="right" /> <br> 
<font size="4"> <b> Eric Fielding, David Bekaert, Heresh Fattahi and Zhang Yunjun with modifications by Franz J Meyer</b> 
<font size="3">  <br>
<font> <b>Date: </b> April 14, 2021 </font>
</font>

<font face="Calibri" size="3">This notebook is a second modification by Eric Fielding from an earlier version of the notebook (https://nbviewer.jupyter.org/github/aria-tools/ARIA-tools-docs/blob/master/JupyterDocs/NISAR/L2_interseismic/mintpySF/smallbaselineApp_aria.ipynb) by David Bekaert, Heresh Fattahi and Zhang Yunjun that was originally focused on San Francisco. 

 This notebook is a modification from the [original](https://nbviewer.jupyter.org/github/insarlab/MintPy-tutorial/blob/main/smallbaselineApp_aria.ipynb) by Heresh Fattahi and Zhang Yunjun. 


**Mapping landslide motion with InSAR time series**

The Caltech-JPL ARIA project in partnership with NASA Getting Ready for NISAR (GRFN) project has been generated surface displacement products (interferograms) mimicking the NISAR L2 GUNW (Geocoded Unwrapped phase interferograms) product formatting. The interferograms are stored at the NASA ASF DAAC, and are accessible with an Open Source set of tools called ARIA-tools. The Miami Insar Timeseries software in PYthon (MintPy), an open-source package for InSAR time-series analysis, is compatible with the outputs from the ARIA-tools package, and in combination with the ARIA-tools pre-processor can be used to estimate ground displacement time-series. 

The Jupyter notebook presented here is meant as an practical example on the use of Jupyter for exploring landslide displacements. In the example below, we will demonstrate a time-series derived from ARIA standard InSAR products over the Los Angeles, California area revealing landslide motion on the Palos Verdes peninsula. 

See the other tutorials on ARIA-tools to learn more about how to use that package.
</font>

<font face="Calibri" size="3">
<div class="alert alert-warning">
<b>To save time, we have pre-ran the ARIA-tools package and data loading into MintPy</b> 

```!ariaDownload.py -b '33.5 34.5 -119.0 -117.9' --track 71```
    
```!ariaTSsetup.py -f 'products/*.nc' -b '33.65 33.9 -118.45 -118.15' --mask Download```
    
```!smallbaselineApp.py -t LASenDT71.txt --dostep load_data```
    

</div>
</font>

<font face="Calibri" size="3">
<div class="alert alert-warning">
 
<b>The staged data was uploaded to an S3 data bucket in AWS so that they can be easily downloaded into the OpenSARLab:</b>
    
```!aws --region=us-east-1 --no-sign-request s3 cp s3://asf-jupyter-data/Fielding/Stack.zip Stack.zip```

When users are not leveraging openSARlabs, they should start from ARIA-tools as download from S3 will not work.
</div>
</font>

<div class="alert alert-danger">
<font face="Calibri" size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK IS FOR ILLUSTRATIVE PURPOSES ONLY AND DOES NOT INCLUDE A HOMEWORK ASSIGNMENT!</b></font> 
</font>
</div>

# 0. Notebook setup

The two cells below must be run each time the notebook is started to ensure correct set-up of the notebook.

In [ ]:
%%capture
import os
import zipfile

# verify mintpy install is complete:
try:
    import numpy as np
    from mintpy import view, tsview, plot_network, plot_transection, plot_coherence_matrix
except ImportError:
    raise ImportError("Looks like mintPy is not fully installed")

In [ ]:
# define the work directory
work_dir = f"{os.path.abspath(os.getcwd())}/data_LA"
print("Work directory: ", work_dir)
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    print(f'Create directory: {work_dir}')
print(f'Go to work directory: {work_dir}')
os.chdir(work_dir)  
    
if not os.path.isfile('Stack.zip'):
    !aws --region=us-east-1 --no-sign-request s3 cp s3://asf-jupyter-data/Fielding/Stack.zip Stack.zip 
      
# verify if download was succesfull
if os.path.isfile('Stack.zip'):
    with zipfile.ZipFile('Stack.zip', 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())
    print('S3 pre-staged data retrieval was successfull')
else:
    print("Download outside openSarLabs is not supported.\nAs alternative please start from ARIA-tools with the commandline calls provided at the top of this notebook")          


<font face="Calibri" size="3">The following command will download all the ARIA standard products over the LA area, which is 575 products at the time of this writing. This will take more than two hours to complete if the data is not already downloaded.</font>

In [ ]:
# download data for descending track 71 over Los Angeles area
#!ariaDownload.py -b '33.5 34.5 -119.0 -117.9' --track 71

<font face="Calibri" size="3">This ARIA time-series setup would cover the whole Los Angeles area and would take a while to process, so we are skipping this setup.</font>

In [ ]:
#!ariaTSsetup.py -f 'products/*.nc' -b '33.5 34.5 -119.0 -117.9' --mask Download

<font face="Calibri" size="3">The following ARIA time-series setup `ariaTSsetup.py` extracts the data that covers only a small area around the Palos Verdes peninsula southwest of Los Angeles to speed the time-series processing, which we specify with the bounding box. We also download the water mask to avoid using data over the ocean.</font>

In [ ]:
#!ariaTSsetup.py -f 'products/*.nc' -b '33.65 33.9 -118.45 -118.15' --mask Download

# 1. smallbaselineApp.py overview

<font face="Calibri" size="3">This application provides a workflow which includes several steps to invert a stack of unwrapped interferograms and apply different corrections to obtain ground displacement timeseries.  
The workflow consists of two main blocks:

* correcting unwrapping errors and inverting for the raw phase time-series (blue ovals),
* correcting for noise from different sources to obtain the displacement time-series (green ovals).

Some steps are optional, which are switched off by default (marked by dashed boundaries). Configuration parameters for each step are initiated with default values in a customizable text file: [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg). In this notebook, we will walk through some of these steps, for a complete example see the [MintPy repository](https://github.com/insarlab/MintPy).

<p align="left">
  <img width="600" src="NotebookAddons/MintPyWorkflow.jpg">
</p>     
<p style="text-align: center;">
    (Figure from Yunjun et al., 2019)
</p>
</font>

## 1.1 Processing Steps of smallbaselineApp.py

<font face="Calibri" size="3">The MintPy **smallbaselineApp.py** application provides a workflow to invert a stack of unwrapped interferograms and apply different (often optional) corrections to obtain ground displacement timeseries. A detailed overview of the options can be retrieved by involking the help option:</font>

In [ ]:
!smallbaselineApp.py --help

## 1.2 Configuring Processing Parameters

<font face="Calibri" size="3">The processing parameters for the **smallbaselineApp.py** are controlled through a configuration file. If no file is provided the default [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg) configuration is used. Here we use `LASenDT71.txt`, which already constains selected, manually modified configuration parameters for this time-series analysis.</font>

# 2. Small Baseline Time Series Analysis

## 2.1. Loading ARIA data into MintPy

<font face="Calibri" size="3">The [ARIA-tools package](https://github.com/aria-tools/ARIA-tools) is used as a pre-processor for MintPY. It has a download tool that wraps around the ASF DAAC API, and includes tools for stitching/cropping and time-series preparation. The output of the time-series preparation is compatible with the [data directory](https://mintpy.readthedocs.io/en/latest/dir_structure/) structure from MintPy. To save time, we have already pre-ran these steps. The commands used were:

```!ariaDownload.py -b '33.5 34.5 -119.0 -117.9' --track 71```

```!ariaTSsetup.py -f 'products/*.nc' -b '33.65 33.9 -118.45 -118.15' --mask Download```
</font>

<font face="Calibri" size="3">The `ariaTSsetup.py` step above (or the pre-processed Stack.zip) extracted the data for the subset we specified and found a total of 439 products that cover our study area. Now we load the data for the subset area into MintPy.</font>

In [ ]:
# define the MintPy time-series directory
mint_dir = work_dir+'/MintPy'
print("MintPy directory: ", mint_dir)
if not os.path.isdir(mint_dir):
    os.makedirs(mint_dir)
    print('Create directory: {}'.format(mint_dir))

# copy the configuration file
os.chdir(work_dir)  
!cp LASenDT71.txt MintPy

print('Go to work directory: {}'.format(mint_dir))
os.chdir(mint_dir)  

!smallbaselineApp.py LASenDT71.txt --dostep load_data

<font face="Calibri" size="3">The output of the loading step is an "inputs" directory containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata.  
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.). 
</font>

In [ ]:
!ls inputs

<font face="Calibri" size="3">
<div class="alert alert-info">
<b>info.py :</b> 
To get general infomation about a MintPy product, run info.py on the file.   
</div>
</font>

In [ ]:
!info.py inputs/ifgramStack.h5

In [ ]:
!info.py inputs/geometryGeo.h5

## 2.2. Plotting the Interferogram Network

<font face="Calibri" size="3">Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- ifgramStack_coherence_spatialAvg.txt: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- Network.pdf: Displays the network of interferograms on time-baseline coordinates, colorcoded by average coherence of the interferograms. 
- CoherenceMatrix.pdf shows the average coherence pairs between all available pairs in the stack.
</font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt  --dostep modify_network
plot_network.main(['inputs/ifgramStack.h5'])


## 2.3.  Mask Generation

<font face="Calibri" size="3">Mask files can be can be used to mask pixels in the time-series processing. Below we generate a mask file based on the connected components, which is a metric for unwrapping quality.</font>

In [ ]:
!generate_mask.py  inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update
view.main(['maskConnComp.h5'])

In [ ]:
#!view.py 

## 2.4 Reference Point Selection


<font face="Calibri" size="3">The interferometric phase is a relative observation by nature. The phases of each unwrapped interferogram are relative with respect to an arbitrary pixel. Therfore we need to reference all interferograms to a common reference pixel.
The step "reference_point" selects a common reference pixel for the stack of interferograms. The default approach of mintpy is to choose a pixel with highest spatial coherence in the stack. Other options include specifying the longitude and latitude of the desired reference pixel or the line and column number of the refence pixel.</font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep reference_point

<font face="Calibri" size="3">Running the "reference_step" adds additional attributes "REF_X, REF_Y" and "REF_LON, REF_LAT" to the ifgramStack.h5 file. To see the attributes of the file run info.py:</font>

In [ ]:
!info.py inputs/ifgramStack.h5 | egrep 'REF_'

<font face="Calibri" size="3">In this case, I set the reference point latitude and longitude to be in a location close to the Portuguese Bend Landslide, and MintPy calculated the X and Y locations.</font>

## 2.5 Inverting the Small Baseline Network

<font face="Calibri" size="3">In the next step we invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date. By default mintpy selects the first acquisition. The estimated time-series is converted to distance change from radar to target and is provided in meters. </font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep invert_network

<font face="Calibri" size="3">The timeseries file contains three datasets:
- the "time-series" which is the interferometric range change for each acquisition relative to the reference acquisition,
- the "date" dataset which contains the acquisition date for each acquisition,
- the "bperp" dataset which contains the timeseries of the perpendicular baseline.  
</font>

## 2.6 Estimating the Long-Term Velocity Rate

<font face="Calibri" size="3">The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation, which is the slope of linear fit to the time-series. </font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep velocity

In [ ]:
scp_args = 'velocity.h5 velocity -v -1 1'
view.main(scp_args.split())

<font face="Calibri" size="3">
<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
    The line of sight (LOS) for this descending Sentinel-1 track is up and east from ground to radar.
</div>
</font>

<font face="Calibri" size="3"><b>Obvious features in the estimated velocity map:</b>

1) There are several features with significant velocity in this area. <br><br>

2) The negative LOS feature on the Palos Verdes peninsula (center left of map) is the Portuguese Bend Landslide, moving down and southwest toward the sea.

3) There are areas of positive and negative LOS change in the area of Long Beach (east part of map). These are due to the extraction of oil and injection of water in oilfields beneath the city and out in the harbor.

4) The block box at 33.76N, 118.36W is the reference pixel for this map, just north of the Portuguese Bend Landslide. 
</font>

# 3. Error Analysis (Separating Signal from Noise)


<font face="Calibri" size="3">Uncertainty of the ground displacement products derived from InSAR time-series, depends on the quality of the inversion of the stack of interferograms and the accuracy in separating the ground displacement from other components of the InSAR data. Therefore the definition of signal vs noise is different at the two main steps in mintpy:  

1) During the inversion: 
    At this step all systematic components of the interferometric phase (e.g., ground displacement, propagation delay, geometrical residuals caused by DEM or platform's orbit inaccuracy) are considered signal, while the interferometric phase decorrelation, phase unwrapping error and phase inconsistency are considered noise. 
    
2) After inversion: the ground displacement component of the time-serieses is signal, and everything else (including the propagation delay and geometrical residuals) are considered noise

Therefore we first discuss the possible sources of error during the inversion and the existing ways in MintPy to evaluate the quality of inversion and to improve the uncertainty of the inversion. Afterwards we explain the different components of the time-series and the different processing steps in MintPy to separate them from ground displacement signal.  
</font>

## 3.1 Quality of the Inversion

<font face="Calibri" size="3">The main sources of noise during the time-series inversion includes decorrelation, phase unwrapping error and the inconsistency of triplets of interferofgrams. Here we mainly focus on the decorrelation and unwrapping errors. We first show the existing quantities in MintPy to evaluate decorrelation and unwrapping errors and then discuss the existing ways in MintPy to reduce the decorrelation and unwrapping errors on the time-series inversion.</font>

### 3.1.1 Average Spatial Coherence

<font face="Calibri" size="3">Mintpy computes temporal average of spatial coherence of the entire stack as a potential ancillary measure to choose reliable pixels after time-series inversion. </font>

In [ ]:
view.main(['avgSpatialCoh.h5'])

### 3.1.2 Temporal Coherence

<font face="Calibri" size="3">In addition to timeseries.h5 which contains the time-series dataset, invert_network produces other quantities, which contain metrics to evaluate the quality of the inversion including temporalCoherence.h5. Temporal coherence represents the consistency of the timeseries with the network of interferograms. 

Temporal coherence varies from 0 to 1. Pixels with values closer to 1 are considered reliable and pixels with values closer to zero are considered unreliable. For a dense network of interferograms, a threshold of 0.7 may be used (Yunjun et al, 2019). Here we are using a threshold of 0.5.</font>

In [ ]:
view.main(['temporalCoherence.h5'])

<font face="Calibri" size="3">With both the spatial coherence and temporal coherence, we can see that the InSAR in the ports of Long Beach and Los Angeles have unstable phase, and the InSAR measurements there will be low quality.</font>

## 3.2. Velocity Error Analysis

<font face="Calibri" size="3">The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

**Mintpy supports additional corrections in its processing not included in this demo:**
- Unwrapping error correction
- Tropospheric delay correction
- deramping
- Topographic residual correction
- Residual RMS for noise evaluation
- Changing the reference date
</font>

In [ ]:
scp_args = 'velocity.h5 velocityStd -v 0 0.2'
view.main(scp_args.split())

<font face="Calibri" size="3">Note that the plot above is the velocity error, not the velocity. The errors generally increase with distance from the reference point and also increase for points with elevations different from the reference point because of topographically correlated water vapor variations that are especially strong in this area.</font>

# 4. Plotting a Landslide Motion Transect 

In [ ]:
scp_args = 'velocity.h5 --start-lalo 33.75 -118.38 --end-lalo 33.72 -118.3 '
plot_transection.main(scp_args.split())

!smallbaselineApp.py smallbaselineApp.cfg --dostep google_earth

<font face="Calibri" size="3">On this transect, the Portuguese Bend Landslide has an average velocity over the last five years that reaches more than 2.5 cm/year in the descending track radar LOS direction (note that the "zero" is about -5 so we have to subtract that first). By analyzing the velocity on the ascending track and combining the two LOS directions, then we could find out that the displacments are largely horizontal, westward (and southward that we can't see with InSAR).</font>

# 5. Reference Material

<font face="Calibri" size="3">

- Original Notebook withe detailed description by Yunjun and Fattahi at: https://nbviewer.jupyter.org/github/insarlab/MintPy-tutorial/blob/master/smallbaselineApp_aria.ipynb

- Mintpy reference: *Yunjun, Z., H. Fattahi, F. Amelung (2019), Small baseline InSAR time series analysis: unwrapping error correction and noise reduction, preprint doi:[10.31223/osf.io/9sz6m](https://eartharxiv.org/9sz6m/).*

- University of Miami online time-series viewer: https://insarmaps.miami.edu/

- Mintpy Github repository: https://github.com/insarlab/MintPy

- ARIA-tools Github Repository: https://github.com/aria-tools/ARIA-tools
</font>

# 6. Change Log

<font face="Calibri" size="2"> <i>LosAngeles_time_series.ipynb - Version 1.2 - April 2021
<br>
    <b>Version Changes</b>
    <ul>
        <li>Code updates to update to latest MintPy version</li>
        <li>Spelling errors and formatting</li>
        <li>Add --region=us-east-1 --no-sign-request to aws s3 cp call</li>
        <li>Capture matplotlib deprecation warnings in import cell</li>
    </ul></i>
    <br>
    Note: matplotlib deprecation and syntax warnings now appear when calling MintPy scripts. We cannot handle or suppress these in the notebook and they should be ignored.
</font>